In [1]:
#!pip install -r requirements.txt
#!git clone https://github.com/iEdumx/train_sadord.git
%cd train_sadord/
!ls -l
#!pip3 freeze > requirements.txt

/home/eduardo/Documentos/Sadord/train_sadord
total 56
drwxr-xr-x 4 eduardo eduardo  4096 nov 10 00:03  data
drwxr-xr-x 2 eduardo eduardo  4096 nov 10 01:19  modelo
-rw-r--r-- 1 eduardo eduardo   617 nov 10 00:03  predict.py
-rw-r--r-- 1 eduardo eduardo  3323 nov 10 01:24  requirements.txt
-rw-r--r-- 1 eduardo eduardo  2305 nov 10 00:03  train.py
-rw-r--r-- 1 eduardo eduardo 32783 nov 10 00:03 'TransferLearning(VGG16).ipynb'


In [2]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K


In [3]:
K.clear_session()

In [4]:
data_entrenamiento = './data/train'
data_validacion = './data/test'

In [5]:
epocas=2
longitud, altura = 150, 150
batch_size = 20
pasos = 2
pasos_validacion = 5
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 2
lr = 0.0005

In [6]:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [7]:
validacion_datagen = ImageDataGenerator(
    rescale=1. / 255)

In [8]:
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 70 images belonging to 2 classes.


In [9]:
imagen_validacion = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 70 images belonging to 2 classes.


In [10]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same", activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

In [12]:
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

In [13]:
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

In [14]:
cnn.fit_generator(
    imagen_entrenamiento,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=imagen_validacion,
    validation_steps=pasos_validacion)

Epoch 1/2
2/2 [==============================] - 16s 8s/step - loss: 2.2959 - acc: 0.5000 - val_loss: 0.8915 - val_acc: 0.5667
Epoch 2/2
2/2 [==============================] - 13s 7s/step - loss: 1.7754 - acc: 0.5000 - val_loss: 2.2449 - val_acc: 0.4333


In [15]:
dir = './modelo/'

if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')